# Evalute whether there appears to be Neural Collapse in policy

In [ ]:
from gymnasium.experimental.wrappers import RecordVideoV0
from orbax.checkpoint import PyTreeCheckpointer, CheckpointManager
from pprint import pprint

import _pickle as pickle
import jax
import json
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import timeit

from jaxl.buffers import get_buffer
from jaxl.constants import *
from jaxl.models import (
    get_model,
    get_policy,
    policy_output_dim,
)
from jaxl.models.policies import MultitaskPolicy
from jaxl.envs import get_environment
from jaxl.envs.rollouts import EvaluationRollout
from jaxl.utils import (
    set_seed,
    parse_dict,
    set_dict_value,
    get_dict_value,
    RunningMeanStd,
    get_device,
)

In [ ]:
run_seed = 0
device = "gpu:0"
get_device(device)
set_seed(run_seed)

In [ ]:
def get_env(agent_path):
    agent_config_path = os.path.join(agent_path, "config.json")
    with open(agent_config_path, "r") as f:
        agent_config_dict = json.load(f)
    return agent_config_dict["learner_config"]["env_config"]

In [ ]:
base_path = "/home/bryanpu1/projects/rl_nc_representation/jaxl"
log_path = os.path.join(base_path, "jaxl/logs")
project_name = "frozen_lake"
run_name = (
    "ppo-02-27-24_14_49_37-b920ed38-5fdf-42de-b1a8-200a54882a14"
)

agent_path = (
    agent_to_load_env_path
) = os.path.join(
    log_path,
    project_name,
    run_name,
)
trained_env_parameters = get_env(agent_to_load_env_path)

num_episodes = 10
env_seed = 9999
buffer_size = 0
num_seeds = 50
record_video = False
exp_name = "-".join(run_name.split("-")[:-8])

In [ ]:
def get_config(agent_path):
    agent_config_path = os.path.join(agent_path, "config.json")
    with open(agent_config_path, "r") as f:
        agent_config_dict = json.load(f)
        agent_config_dict["learner_config"]["env_config"] = trained_env_parameters
        agent_config_dict["learner_config"]["env_config"]["env_kwargs"][
            "render_mode"
        ] = "rgb_array"
        if "policy_distribution" not in agent_config_dict["learner_config"]:
            agent_config_dict["learner_config"][
                "policy_distribution"
            ] = CONST_DETERMINISTIC
        set_dict_value(agent_config_dict, "vmap_all", False)
        (multitask, num_models) = get_dict_value(agent_config_dict, "num_models")
        agent_config = parse_dict(agent_config_dict)
    return agent_config, {
        "multitask": multitask,
        "num_models": num_models,
    }

In [ ]:
checkpoint_manager = CheckpointManager(
    os.path.join(agent_path, "models"),
    PyTreeCheckpointer(),
)
params = checkpoint_manager.restore(checkpoint_manager.latest_step())
model_dict = params[CONST_MODEL_DICT]
agent_policy_params = model_dict[CONST_MODEL][CONST_POLICY]
agent_obs_rms = False
if CONST_OBS_RMS in params:
    agent_obs_rms = RunningMeanStd()
    agent_obs_rms.set_state(params[CONST_OBS_RMS])

agent_config, aux = get_config(agent_path)
env = get_environment(agent_config.learner_config.env_config)

if record_video:
    env = RecordVideoV0(env, f"videos/{exp_name}-videos")

input_dim = env.observation_space.shape
output_dim = policy_output_dim(env.act_dim, agent_config.learner_config)
model = get_model(
    input_dim,
    output_dim,
    getattr(agent_config.model_config, "policy", agent_config.model_config),
)
policy = get_policy(model, agent_config.learner_config)
if aux["multitask"]:
    policy = MultitaskPolicy(policy, model, aux["num_models"])

agent_rollout = EvaluationRollout(env, seed=env_seed)
agent_rollout.rollout(
    agent_policy_params, policy, agent_obs_rms, num_episodes, None
)